<a href="https://colab.research.google.com/github/dmosh/deeplearning/blob/main/imageclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
#importing the neccessary libraries
import glob
import cv2 as cv
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
import os
import requests
import urllib
from io import BytesIO
from PIL import Image
import pathlib
from sklearn.metrics import confusion_matrix
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [11]:
#load and read the images
url = "https://github.com/dmosh/deeplearning/tree/main/dataset1"
image_dir = tf.keras.utils.get_file("dataset1", origin=url, untar=False)
image_dir = pathlib.Path(image_dir)